In [8]:
import splitfolders  
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [ ]:
original_dataset_dir = "./dataset"
output_dir = "./data-split"

# Split dataset into train (70%), val (20%), test (10%) with stratified sampling
splitfolders.ratio(
    original_dataset_dir,
    output=output_dir,
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    group_prefix=None,  # Ensures similar prefixes stay in the same set
    move=False,
)

In [9]:

# Define paths
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Image dimensions and parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4


In [10]:

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)  

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Ensure reproducibility for evaluation
)

Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Found 74 images belonging to 4 classes.


In [14]:

# Load MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Compile model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train frozen model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,

    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.2104 - loss: 1.9931 - val_accuracy: 0.2409 - val_loss: 1.4638 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3212 - loss: 1.5998 - val_accuracy: 0.3869 - val_loss: 1.3433 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.3259 - loss: 1.4796 - val_accuracy: 0.4015 - val_loss: 1.2724 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 998ms/step - accuracy: 0.4359 - loss: 1.2747 - val_accuracy: 0.4672 - val_loss: 1.2052 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 988ms/step - accuracy: 0.3911 - loss: 1.3125 - val_accuracy: 0.4672 - val_loss: 1.1706 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.5231 - loss: 1.1080 - val_accuracy: 0.4745 - val_loss: 1.1602 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 15s 960ms/step - accuracy: 0.459

In [16]:
# Fine-tune the base model
base_model.trainable = True

# Optionally freeze the earlier layers to prevent overfitting
for layer in base_model.layers[:60]:  # Adjust based on experimentation
    layer.trainable = False

# Re-compile for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.4560 - loss: 1.2670 - val_accuracy: 0.5693 - val_loss: 1.0377 - learning_rate: 1.0000e-05
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4970 - loss: 1.2336 - val_accuracy: 0.5474 - val_loss: 1.0752 - learning_rate: 1.0000e-05
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5710 - loss: 0.9770 - val_accuracy: 0.5109 - val_loss: 1.1164 - learning_rate: 1.0000e-05
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5461 - loss: 1.0731 - val_accuracy: 0.5255 - val_loss: 1.1454 - learning_rate: 1.0000e-05
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5746 - loss: 0.9831 - val_accuracy: 0.5182 - val_loss: 1.1471 - learning_rate: 2.0000e-06


In [18]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 347ms/step - accuracy: 0.5299 - loss: 1.1667
Test Accuracy: 0.57


In [19]:
# Confusion Matrix and Classification Report
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predictions
y_pred = np.argmax(model.predict(test_generator), axis=-1)
print("Confusion Matrix:")
print(confusion_matrix(test_generator.classes, y_pred))
print("\nClassification Report:")
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step  
Confusion Matrix:
[[11  0  4  3]
 [ 4  3  6  6]
 [ 0  0 18  0]
 [ 3  2  4 10]]

Classification Report:
              precision    recall  f1-score   support

       Angry       0.61      0.61      0.61        18
        Fear       0.60      0.16      0.25        19
       Happy       0.56      1.00      0.72        18
         Sad       0.53      0.53      0.53        19

    accuracy                           0.57        74
   macro avg       0.57      0.57      0.53        74
weighted avg       0.57      0.57      0.52        74

